# CALCULATE SERVICE COVERAGE OF BUS NETWORK

In [1]:
import yaml
import pandas as pd
import numpy as np
from lxml import etree

# Set up Plan and Schedule

### Before scenario

In [2]:
PLAN_PATH = "data\\before\\plans_scale0.375true.xml"
SCHEDULE_PATH = "data\\before\\output_transitSchedule.xml"

### After Scenario

In [3]:
# PLAN_PATH = "data\\after\\plans_scale0.375true.xml"
# SCHEDULE_PATH = "data\\after\\output_transitSchedule.xml"

### Simple and Real Scenario

In [4]:
# PLAN_PATH = "data\\real\\plans_scale0.375true.xml"
# SCHEDULE_PATH = "data\\real\\transit_schedule.xml"

# PLAN_PATH = "data\\simple_scenario\\plans.xml"
# SCHEDULE_PATH = "data\\simple_scenario\\schedule.xml"

## head plan file

In [5]:
import os
print("Thư mục hiện tại:", os.getcwd())
print("Danh sách file/thư mục tại đây:", os.listdir())

Thư mục hiện tại: d:\CONG_VIEC_VTS\01-WORKS\02-MATSIM\01-CHAM-DIEM-MANG-LUOI-XE-BUS\03-Code\2-PERSONAL\v2_Danh_Gia_Tong_The_MangLuoiXeBus
Danh sách file/thư mục tại đây: ['.git', '.gitignore', '.venv', 'AI_fix_estimate_impact_to_OD.ipynb', 'BusNetScoreCalculator.kt', 'BusPassengerHandler.kt', 'bus_map_offset_full_v4.png', 'bus_map_offset_v3.png', 'bus_network_map.png', 'conf', 'data', 'distance_and_map_bus_route.ipynb', 'estimate_impact_to_OD.ipynb', 'gen_dummy_scenario.py', 'otp.ipynb', 'README.md', 'requirements.txt', 'Result_After_Before.md', 'ridership.ipynb', 'service_coverage.ipynb', 'src', 'temp', 'travel_time.ipynb', 'Điểm của Hải.txt']


# Get home coordinate

In [6]:
!powershell -Command "Get-Content 'data\real\plans_scale0.375true.xml' -TotalCount 20 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

5: <?xml version="1.0" encoding="utf-8"?>
5: <!DOCTYPE population SYSTEM "http://www.matsim.org/files/dtd/population_v5.dtd">
5: 
5: <population>
5: 
5: <!-- ====================================================================== -->
5: 
5: 	<person id="12052000_12052000_0">
5: 		<plan selected="yes">
5: 			<act type="home" x="454788.14217382803" y="5735115.491572791" end_time="07:16:20" />
5: 			<leg mode="car">
5: 			</leg>
5: 			<act type="work" x="451947.00656881003" y="5736160.56555245" end_time="14:55:41" />
5: 			<leg mode="car">
5: 			</leg>
5: 			<act type="home" x="454788.14217382803" y="5735115.491572791" end_time="16:22:14" />
5: 			<leg mode="car">
5: 			</leg>
5: 			<act type="shopping" x="455006.52115909086" y="5734909.411959048" end_time="17:41:13" />
5: 			<leg mode="car">


In [7]:
!@REM !powershell -Command "Get-Content 'data\simple_scenario\plans.xml' -TotalCount 20 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"@REM 

### Class lưu dữ liệu home

In [8]:
class HomeCoordinate:
    def __init__(self, person_id: str, x: float, y: float):
        self.person_id = person_id
        self.x = x
        self.y = y



In [9]:
parser = etree.XMLParser(remove_blank_text=True)
tree = etree.parse(PLAN_PATH, parser)
root = tree.getroot()


### Lọc các tọa độ home của mỗi người

In [10]:
# print(len(root.xpath('//population/person/plan[@selected="yes"]/act[@type="home"][1]')))

# xpath luôn trả ra list
# /tag (Ở đầu)	Tìm trong Root (Gốc) - Luôn bắt đầu tìm từ đỉnh cao nhất của file XML trar ra các list có tag tương ứng.
# //tag (Ở đầu): 	Tìm trong toàn bộ file - Tìm tất cả các node khớp với tên đó, bất kể nó nằm ở đâu.
# ./tag hoặc Không có xuyệt: Relative (Tương đối) - Chỉ tìm bên trong node bạn đang đứng.
# @color là tên attribute
# [@attr='value']	Lọc theo attribute - Chỉ chọn các node có attribute là value. Trả ra 1 list

home_coordinates: list[HomeCoordinate] = []
residence_count = len(root.xpath('//population/person'))
print("Residence count: ", residence_count)

for node in root.xpath('//population/person'):
    id = node.xpath('@id')[0]
    act = node.xpath(('./plan[@selected = "yes"]/act[@type="home"]'))
    x = act[0].xpath('@x')[0]
    y = act[0].xpath('@y')[0]
    print([id,x,y])
    home_coordinates.append(HomeCoordinate(id, float(x), float(y)))
    
print("Home location count: ", len(home_coordinates))
    

Residence count:  22752
['12052000_12052000_0', '454788.14217382803', '5735115.491572791']
['12052000_12052000_1', '455903.06702539005', '5730103.746178518']
['12052000_12052000_10', '453482.5329881937', '5737532.2543261545']
['12052000_12052000_100', '454028.73456912086', '5736860.826699184']
['12052000_12052000_1000', '453714.23944590375', '5734294.511748257']
['12052000_12052000_1001', '451629.12612927984', '5734252.037979559']
['12052000_12052000_1002', '454567.7673419709', '5735269.102628146']
['12052000_12052000_1003', '454304.5904565242', '5733044.211043378']
['12052000_12052000_1004', '454114.68423825584', '5736705.892827102']
['12052000_12052000_1005', '455605.6635947757', '5728393.139267473']
['12052000_12052000_1006', '455921.60987156024', '5730325.250562869']
['12052000_12052000_1007', '451055.42504563194', '5734951.539055572']
['12052000_12052000_1008', '454635.7365508042', '5740286.18100591']
['12052000_12052000_1009', '453917.57396971586', '5729419.892689186']
['12052000

# Get Bus Stop Coordinate

In [11]:
!powershell -Command "Get-Content 'data\real\transit_schedule.xml' -TotalCount 90 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

5: <?xml version="1.0" encoding="UTF-8"?>
5: <!DOCTYPE transitSchedule SYSTEM "http://www.matsim.org/files/dtd/transitSchedule_v2.dtd">
5: 
5: <transitSchedule>
5: 	<attributes>
5: 		<attribute name="coordinateReferenceSystem" class="java.lang.String">Atlantis</attribute>
5: 	</attributes>
5: 
5: 	<transitStops>
5: 		<stopFacility id="9445451_a" x="455551.5822241595" y="5728935.910032587" linkRefId="6050" isBlocking="false"/>
5: 		<stopFacility id="9445451_b" x="455551.5822241595" y="5728935.910032587" linkRefId="6051" isBlocking="false"/>
5: 		<stopFacility id="9445452_a" x="455344.27894843515" y="5728335.906164524" linkRefId="10112" isBlocking="false"/>
5: 		<stopFacility id="9445452_b" x="455344.27894843515" y="5728335.906164524" linkRefId="10113" isBlocking="false"/>
5: 		<stopFacility id="9445453_a" x="456138.141874901" y="5728333.529178958" linkRefId="3761" isBlocking="false"/>
5: 		<stopFacility id="9445453_b" x="456138.141874901" y="5728333.529178958" linkRefId="3762" isBlockin

In [12]:
!@REM !powershell -Command "Get-Content 'data\simple_scenario\schedule.xml' -TotalCount 90 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

### class lưu dữ liệu stop

In [13]:
class StopCoordinate:
    def __init__(self, stop_id: str, x: float, y: float):
        self.stop_id = stop_id
        self.x = x
        self.y = y


### Lọc các bến trong tuyến có transportmode string 'bus' ( ở dạng hoa hay thường)

In [14]:
parser = etree.XMLParser(remove_blank_text=True)
tree = etree.parse(SCHEDULE_PATH, parser)
root = tree.getroot()

full_stop_coordinate : set[StopCoordinate] = set()
bus_stop_coordinate : set[StopCoordinate] = set()

for node in root.xpath('//transitSchedule/transitStops/stopFacility'):
    id = node.xpath("@id")[0]
    x = node.xpath("@x")[0]
    y = node.xpath("@y")[0]
    print([id, x, y])
    full_stop_coordinate.add(StopCoordinate(id, float(x), float(y)))


['9445451_a', '455551.5822241595', '5728935.910032587']
['9445451_b', '455551.5822241595', '5728935.910032587']
['9445452_a', '455344.27894843515', '5728335.906164524']
['9445452_b', '455344.27894843515', '5728335.906164524']
['9445453_a', '456138.141874901', '5728333.529178958']
['9445453_b', '456138.141874901', '5728333.529178958']
['9445454_a', '455277.26995572867', '5729416.739276236']
['9445454_b', '455277.26995572867', '5729416.739276236']
['9445455_a', '455575.0997953637', '5728081.058691686']
['9445455_b', '455575.0997953637', '5728081.058691686']
['9445457_a', '455324.5209031641', '5728641.08167708']
['9445457_b', '455324.5209031641', '5728641.08167708']
['9445458_a', '456048.08208885347', '5728185.5007801615']
['9445540_a', '452874.05862960796', '5729513.568654206']
['9445542_a', '454059.1543358987', '5729375.385785605']
['9445543_a', '452898.44835668604', '5729762.162964511']
['9445543_b', '452898.44835668604', '5729762.162964511']
['9445543_c', '452882.5049580507', '5729605

### LOGIC: 1 tuyến được cho là bus khi mà transportMode có chứa hint_bus_route

In [15]:
hint_bus_route = ["pt","bus"]


print("----- Bus Stops -----")
for node in root.xpath('//transitSchedule/transitLine/transitRoute'):
    results = node.xpath('transportMode/text()')[0]
    for hint in hint_bus_route:
        if hint in str(results).lower():
            stop_tag_list = node.xpath("./routeProfile/stop")
            for stop in stop_tag_list:
                stop_id = stop.xpath('@refId')[0]
                for full_stop in full_stop_coordinate:
                    if stop_id == full_stop.stop_id:
                        print([stop_id, full_stop.x, full_stop.y])
                        bus_stop_coordinate.add(full_stop)
                        break


----- Bus Stops -----
['9470168_a', 453495.1993624639, 5733382.242049832]
['9470294_b', 453631.54785475414, 5733968.438650811]
['9470269_c', 453858.13192102924, 5734240.441531738]
['9470155_a', 454202.14519041113, 5734290.287219761]
['9470232_a', 454683.654622206, 5734472.43368548]
['9470159_a', 454883.2182515095, 5734191.003274704]
['9470290_a', 455473.7753072552, 5733512.7494956]
['9470118_a', 455805.31874317606, 5733544.821176317]
['9470150_a', 456111.1386780715, 5733669.8330386]
['9470264_a', 456273.0338107613, 5733932.383744901]
['9470127_a', 456465.89360168413, 5734148.118433831]
['9470127_a', 456465.89360168413, 5734148.118433831]
['9470264_b', 456273.0338107613, 5733932.383744901]
['9470150_b', 456111.1386780715, 5733669.8330386]
['9470125_a', 456629.7353463507, 5733325.643643517]
['9470121_a', 456852.0340075999, 5732655.6847743075]
['9470405_a', 456696.9561706177, 5732071.815346967]
['9470123_a', 456441.6162601197, 5732069.153336799]
['9470126_a', 456301.6549054334, 5731918.19

# Calculate Service Coverage

In [16]:
def calculte_service_coverage(home_coor_list: list[HomeCoordinate], stop_coor_list: set[StopCoordinate], radia: float):
    service_coverage = 0
    for home_coor in home_coor_list:
        for stop_coor in stop_coor_list:
            distance = np.sqrt((home_coor.x - stop_coor.x)**2 + (home_coor.y - stop_coor.y)**2)
            if distance <= radia:
                service_coverage += 1
                break
    return service_coverage

service_coverage = calculte_service_coverage(home_coordinates, bus_stop_coordinate, 400)
print(f"Service coverage Bus Stops (400m): ${service_coverage};  percent: {service_coverage/residence_count*100:.2f}%")


Service coverage Bus Stops (400m): $9113;  percent: 40.05%
